In [54]:
import re
import os
import cv2
import csv              
import random
import numpy as np
import pandas as pd
import seaborn as sns
from PIL  import  Image
import matplotlib.pyplot as plt

In [55]:
train=pd.read_csv('../input/global-wheat-detection/train.csv')
sample_output = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')
train_dir = '../input/global-wheat-detection/train/'
test_dir ='../input/global-wheat-detection/test/'

In [56]:
train.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [57]:
df = pd.read_csv('../input/global-wheat-detection/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['xmin', 'ymin', 'w', 'h']):
    df[column] = bboxs[:,i]

df.drop(columns=['bbox'], inplace=True)
df['xmax'] = df['xmin'] + df['w']
df['ymax'] = df['ymin'] + df['h']
# df['x_center'] = df['xmin'] + df['w']/2
# df['y_center'] = df['ymin'] + df['h']/2

df['classes'] = 0
df = df[['image_id','xmin', 'ymin','xmax', 'ymax', 'w', 'h','classes']]
df.head()

,image_id,xmin,ymin,xmax,ymax,w,h,classes
0,b6ab77fd7,834.0,222.0,890.0,258.0,56.0,36.0,0
1,b6ab77fd7,226.0,548.0,356.0,606.0,130.0,58.0,0
2,b6ab77fd7,377.0,504.0,451.0,664.0,74.0,160.0,0
3,b6ab77fd7,834.0,95.0,943.0,202.0,109.0,107.0,0
4,b6ab77fd7,26.0,144.0,150.0,261.0,124.0,117.0,0


In [58]:
# exporting cleaned dataframe
df.to_csv("data.csv")

In [59]:
index = list(set(df.image_id))
# print(index)

# Creating Required annotation file
* each image has corresponding annotation file -[image_id.xml] contain bbox elements
* we will feed the inputs as they were expected no change in Mast R-cnn config

In [67]:
import shutil
try:
    shutil.copytree('../input/global-wheat-detection/train','./images','./images')
except:
  print("error or maybe file Already exsist")

error or maybe file Already exsist


In [68]:
from pathlib import Path
Path("./annots").mkdir(parents=True, exist_ok=True)

In [69]:
f = open('./data.csv')
csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()
print(data[:4])

[['', 'image_id', 'xmin', 'ymin', 'xmax', 'ymax', 'w', 'h', 'classes'], ['0', 'b6ab77fd7', '834.0', '222.0', '890.0', '258.0', '56.0', '36.0', '0'], ['1', 'b6ab77fd7', '226.0', '548.0', '356.0', '606.0', '130.0', '58.0', '0'], ['2', 'b6ab77fd7', '377.0', '504.0', '451.0', '664.0', '74.0', '160.0', '0']]


## XML file conversion code

In [70]:
import xml.etree.ElementTree as gfg 
f = open('./data.csv')
csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

def GenerateXML(annots_dir,img_dir,image_id) :
    fileName = image_id+".xml"
    root = gfg.Element("annotation") 

    b1 = gfg.SubElement(root, "folder")
    b1.text = "wheat_heads"
    b2 = gfg.SubElement(root, "filename") 
    b2.text = image_id+".jpg"
    b3 = gfg.SubElement(root, "path") 
    b3.text = img_dir+image_id+".jpg"
    
    m3 = gfg.Element("size") 
    root.append (m3) 
    
    d1 = gfg.SubElement(m3, "width") 
    d1.text = "1024"
    d2 = gfg.SubElement(m3, "height") 
    d2.text = "1024"
    d3 = gfg.SubElement(m3, "depth") 
    d3.text = "3"

    for row in data[1:] :
        if row[1]== image_id:

            m4 = gfg.Element("object") 
            root.append (m4) 
            
            d1 = gfg.SubElement(m4, "name") 
            d1.text = "wheat_head"
            
            m5=gfg.Element("bndbox")
            m4.append(m5)
            d1 = gfg.SubElement(m5,"xmin")
            d1.text= str(int(float(row[2])))
            d2 = gfg.SubElement(m5,"ymin")
            d2.text= str(int(float(row[3])))
            d3 = gfg.SubElement(m5,"xmax")
            d3.text= str(int(float(row[4])))
            d4 = gfg.SubElement(m5,"ymax")
            d4.text= str(int(float(row[5])))

    tree = gfg.ElementTree(root) 
    
    with open (annots_dir+fileName, "wb") as files :
        tree.write(files)

In [71]:
# os.listdir('./images')

['7f01525b1.jpg',
 '27bf608b8.jpg',
 '94b3764f6.jpg',
 '57c9b9982.jpg',
 '1c6e42234.jpg',
 '13050d5e4.jpg',
 'e142e85ad.jpg',
 'd2ae1e503.jpg',
 'd7e657453.jpg',
 'df1ba5253.jpg',
 '9c72bafb6.jpg',
 'e3a86f7ff.jpg',
 'a28d1eb6d.jpg',
 'd623a5d12.jpg',
 'f43b1b260.jpg',
 '5e0a7fb4d.jpg',
 'e88d43ec5.jpg',
 'ce4124be0.jpg',
 'c808023f3.jpg',
 '51260fee0.jpg',
 '26f09dc76.jpg',
 '6f4d09643.jpg',
 '40d727f36.jpg',
 '4dc07d36a.jpg',
 '18f8b24f1.jpg',
 '5901a73fe.jpg',
 '2ac5f9e85.jpg',
 '0e0809a3d.jpg',
 '49a9d29b6.jpg',
 '24dda66d6.jpg',
 'a9b3451c9.jpg',
 '278038027.jpg',
 'c18c32da0.jpg',
 'c20958f55.jpg',
 'f1b34145a.jpg',
 '119969932.jpg',
 '570ca24af.jpg',
 '901e23e9b.jpg',
 '382bf4ed7.jpg',
 '284a79f05.jpg',
 '59dbb2e28.jpg',
 '12f742390.jpg',
 '5ea4aa57a.jpg',
 '7d0d1f2f4.jpg',
 '3a1c78a92.jpg',
 'bb3a2e25d.jpg',
 '2ccb37ea9.jpg',
 'f15aa967e.jpg',
 '059125072.jpg',
 '3cf295f6c.jpg',
 '4341d37a9.jpg',
 '9bc09acf1.jpg',
 'f57f7edd1.jpg',
 '0eab04c31.jpg',
 '229d8a499.jpg',
 'd3b3b562

In [ ]:
#wait for a long time
for image_id in index:
    GenerateXML("./annots/","./images/", image_id) 

In [ ]:
## count xml files
# os.listdir('./annots')

i=0
for j in (os.listdir('./annots')):
    i+=1
print(i)

In [ ]:
from PIL  import  Image
img = Image.open('./images/7f01525b1.jpg')
plt.imshow(img)

In [ ]:
!git clone https://github.com/mmaithani/Mask_RCNN.git

In [ ]:
cd Mask_RCNN

In [ ]:
!python setup.py install

In [ ]:
!pip install imutils
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
import mrcnn
from mrcnn.utils import Dataset
from mrcnn.model import MaskRCNN
import numpy as np
from numpy import zeros
from numpy import asarray
import colorsys
import argparse
import imutils
import random
import cv2
import os
import time
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from keras.models import load_model
%matplotlib inline
from os import listdir
from xml.etree import ElementTree

In [ ]:
class myMaskRCNNConfig(Config):
    # give the configuration a recognizable name
    NAME = "MaskRCNN_config"
 
    # set the number of GPUs to use along with the number of images
    # per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # number of classes (we would normally add +1 for the background)
     # kangaroo + BG
    NUM_CLASSES = 1+1
   
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 131
    
    # Learning rate
    LEARNING_RATE=0.006
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=10

In [ ]:
config = myMaskRCNNConfig()

In [ ]:
config.display()

In [ ]:
class WheatData(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):

        # Add classes. We have only one class to add.
        self.add_class("dataset", 1, "wheat_head")
        
        # define data locations for images and annotations
        images_dir = './images/'
        annotations_dir = './annots/'
        
        # Iterate through all files in the folder to 
        #add class, images and annotaions
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4]
            
#             # skip bad images
#             if image_id in ['00090']:
#                 continue
#             # skip all images after 150 if we are building the train set
#             if is_train and int(image_id) >= 150:
#                 continue

           # setting image file
            img_path = images_dir + filename
            
            # setting annotations file
            ann_path = annotations_dir + image_id + '.xml'
            
            # adding images and annotations to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
# extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
        
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height
# load the masks for an image
    """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
     """
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        
        # define anntation  file location
        path = info['annotation']
        
        # load XML
        boxes, w, h = self.extract_boxes(path)
       
        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('kangaroo'))
        return masks, asarray(class_ids, dtype='int32')
# load an image reference
#      """Return the path of the image."""
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        print(info)
        return info['path']

In [ ]:
cd ..

In [ ]:
os.getcwd()

In [ ]:
os.listdir(train_dir)

In [ ]:
# prepare train set
train_set = WheatData()
train_set.load_dataset("./", is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_id))

# prepare test/val set
test_set = WheatData()
test_set.load_dataset("./", is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_id))

In [ ]:
print("Loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')

In [ ]:
os.listdir('./Mask_RCNN')

In [ ]:
#load the weights for COCO
model.load_weights('.\\Mask_RCNN\\mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
## train heads with higher lr to speedup the learning
model.train(train_set, test_set, learning_rate=2*config.LEARNING_RATE, epochs=5, layers=’heads’)
history = model.keras_model.history.history